In [9]:
import pandas as pd
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
import collections
warnings.filterwarnings('ignore')

In [6]:
path = 'C:/ZhangLI/Codes/DataSet/新闻推荐比赛数据/'
train = path + 'articles.csv'
train_click = path + 'train_click_log.csv'

In [7]:
# 使用历史浏览 点击文章的数据信息预测用户未来的点击行为，即用户最后一次点击新闻的文章
# 问题转换 30万用户 200w点击 36w篇文章
train_df = pd.read_csv(train)
train_click_df = pd.read_csv(train_click)

In [13]:
train_click_df.memory_usage().sum()/1024**2

76.39788055419922

In [14]:
def reduce_memory(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_types = df[col].dtypes
        if col_types in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_types)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem, 
                                                                                                    100*(start_mem-end_mem)/start_mem, 
                                                                                                    (time.time()-starttime)/60))
    return df

In [17]:
# 定义
user_recall_items_dict = collections.defaultdict(dict)
user_recall_items_dict

defaultdict(dict, {})

In [ ]:
get_user_item_time()

In [ ]:
# 根据点击时间获取用户的点击文章序列 {user1: [(item1, time1), (item2, item2)..]..}
def get_user_item_time(click_df):
    